In [18]:
import anthropic
from dotenv import load_dotenv
from datasets import load_from_disk
import pandas as pd

load_dotenv()

True

In [2]:
dataset_dict = load_from_disk("../../data/complete_dataset/")

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})


In [23]:
import pandas as pd

validation_df = pd.DataFrame(dataset_dict['validation'])
df = validation_df[['answer_urdu']].copy()
df['id'] = range(len(df))
df.rename(columns={'answer_urdu': 'answers'}, inplace=True)
df = df[['id', 'answers']]

display(df)

,id,answers
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...
...,...,...
119,119,وزیر اعظم مشترکہ مفادات کی کونسل کا چیئرمین ہو...
120,120,نہیں، کسی شخص کو شہادت حاصل کرنے کی غرض سے اذی...
121,121,سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے مساوی عہدے...
122,122,مالی بل کی ابتدا قومی اسمبلی میں ہوگی۔


In [3]:
def get_urdu_response(context, question):
    client = anthropic.Anthropic()
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        temperature=0.0,
        max_tokens=1024,
        system="You are a Pakistani legal expert who speaks in Urdu. You only respond in Urdu script.",
        messages=[ 
            {"role": "user", "content": f"""Below is a context provided, followed by a question. Your task is to answer the question based on the given context. The answer should consist only of the response to the question in Urdu and should not include any other information or commentary.

    Context: ```{context}```

    Question: ```{question}```

    Answer:"""}
        ]
    )
    
    return message.content[0].text

In [7]:
from tqdm import tqdm

predicted_answers = []

for i in tqdm(range(len(dataset_dict['validation'])), desc="Processing responses", unit="item"):
    test_point = dataset_dict['validation'][i]
    
    context = test_point['context_urdu']
    question = test_point['question_urdu']
    
    predicted_answer = get_urdu_response(context=context, question=question)
    
    predicted_answers.append(predicted_answer)

Processing responses: 100%|██████████| 124/124 [05:12<00:00,  2.52s/item]


In [12]:
predicted_answers[:5]

['اسلامی کونسل کے رکن کی مدت کار تین سال ہوتی ہے۔',
 'پاکستان میں یوم آغاز سے قبل درج ذیل علاقوں کو قبائلی علاقے جات سمجھا جاتا تھا:\n\n۱۔ بلوچستان اور خیبر پختونخوا کے قبائلی علاقے جات۔\n۲۔ امب، چترال، دیر اور سوات کی سابقہ ریاستیں۔',
 'نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی کے بغیر کسی دوسری عدالت عالیہ میں تبادلہ نہیں کیا جا سکتا۔ صدر کسی جج کا تبادلہ صرف اس کی رضامندی سے ہی کر سکتا ہے، اور یہ تبادلہ چیف جسٹس پاکستان اور دونوں متعلقہ عدالت عالیہ کے چیف جسٹسوں کے مشورے سے ہی کیا جا سکتا ہے۔',
 'وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت کے پاس موجودہ قواعد و احکام میں ترمیم کرنے کا اختیار ہے۔',
 'پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے سے پہلے مذکورہ رکن کو اس بارے میں اظہار وجوہ کا موقع فراہم کرنا چاہیے کہ کیوں نہ اس کے خلاف مذکورہ اعلان کر دیا جائے۔']

In [24]:
df['prediction_text_claude'] = predicted_answers

In [28]:
display(df.head())

,id,answers,prediction_text_claude
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...,اسلامی کونسل کے رکن کی مدت کار تین سال ہوتی ہے۔
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...,پاکستان میں یوم آغاز سے قبل درج ذیل علاقوں کو ...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...,پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے ...


In [26]:
df.to_csv('../../data/model_qa_responses/claude-3.5-sonnet-responses.csv', index=False)